In [ ]:
# add rough markdowns 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from google.cloud import bigquery

Sections to add
- Basic Exploration of `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
- Pandas Profiling of the data `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
- Check CE's with skill restrictions and data sizes 
- Audience Profiling - Python/BQ?
- Why do only for US? Need proper/business justification for removing/filtering data*

In [2]:
# Creating a table with unnested data that can be used further 
query_create_newtable = """
CREATE table ecomm_unnest_purchase.ecomm_raw_purchase_version03
as
with main_data_purchase as 
(
SELECT
    event_date,event_timestamp,
    event_name,event_value_in_usd,
    user_pseudo_id,
    events.key as key,
    events.value.string_value as value_string,
    events.value.int_value as value_int,
    user_first_touch_timestamp,
    user_ltv.revenue,
    user_ltv.currency,device.category,
    device.mobile_brand_name,device.mobile_model_name,
    device.operating_system,device.language,
    device.is_limited_ad_tracking,
    device.web_info.browser,device.web_info.browser_version,
    geo.continent,geo.sub_continent,geo.country,geo.region,
    ecommerce.transaction_id,
    item.item_id,item.item_name,
    item.item_brand,item.item_variant,
    item.item_category,
    item.price_in_usd,item.price,item.quantity,
    item.item_revenue_in_usd,
    
FROM 
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
    UNNEST(event_params) as events,
    UNNEST(items) as item
WHERE 
    event_name = "purchase"
    AND 
    key IN ('transaction_id','engagement_time_msec','medium','payment_type',
            'ga_session_number','session_engaged','campaign',
            'value','value','shipping_tier')
)
SELECT user_pseudo_id,event_timestamp,
    MAX(revenue) as revenue,
    MAX(currency) as currency,
    MAX(category) as category,
    MAX(mobile_brand_name) as mobile_brand_name, 
    MAX(operating_system) as operating_system,
    MAX(language) as language,
    MAX(browser) as browser,
    Max(browser_version) as browser_version,
    MAX(continent) as continent,
    MAX(sub_continent) as sub_continent,
    MAX(country) as country,
    MAX(region) as region,
    MAX(item_id) as item_id,
    MAX(item_name) as item_name,
    MAX(item_brand) as item_brand,
    MAX(item_variant) as item_variant,
    MAX(item_category) as item_category,
    MAX(price_in_usd) as price_in_usd,
    MAX(price) as price,
    MAX(quantity) as quantity,
    MAX(item_revenue_in_usd) as item_revenue_in_usd,
    MAX(IF(key = "transaction_id", value_string, NULL)) AS transaction_id,
    MAX(IF(key = "campaign", value_string, NULL)) AS campaign,
    MAX(IF(key = "engagement_time_msec", value_int, NULL)) AS engagement_time_msec,
    MAX(IF(key = "payment_type", value_string, NULL)) AS payment_type,
    MAX(IF(key = "session_engaged", value_int, NULL)) AS session_engaged,
    MAX(IF(key = "medium", value_string, NULL)) AS medium,
    MAX(IF(key = "shipping_tier", value_string, NULL)) AS shipping_tier, 
    MAX(IF(key = "ga_session_number", value_int, NULL)) AS ga_session_number,
FROM 
    main_data_purchase
GROUP BY 
    user_pseudo_id,event_timestamp
    """

client = bigquery.Client()
query_job = client.query(query_create_newtable)

In [4]:
data_purchase_uniquet_id = """
SELECT *
from ecomm_unnest_purchase.ecomm_raw_purchase_version03
WHERE transaction_id is not NULL
"""
query_job = client.query(data_purchase_uniquet_id)
data_purchase_uniquet_id_df = query_job.to_dataframe()
data_purchase_uniquet_id_df.head(5)

,user_pseudo_id,event_timestamp,revenue,currency,category,mobile_brand_name,operating_system,language,browser,browser_version,continent,sub_continent,country,region,item_id,item_name,item_brand,item_variant,item_category,price_in_usd,price,quantity,item_revenue_in_usd,transaction_id,campaign,engagement_time_msec,payment_type,session_engaged,medium,shipping_tier,ga_session_number
0,74275572.8018336190,1607197996525068,156.0,USD,desktop,Google,Web,None,Chrome,87.0,Asia,Western Asia,Iraq,(not set),9196407,Keyboard DOT Sticker,Google,Single Option Only,Uncategorized Items,44.0,44.0,1,44.0,5945714572,None,1.0,Pay with credit card,NaN,None,FedEx Ground,2
1,2994751.6580189667,1608520816198630,183.0,USD,desktop,Google,Web,en,Chrome,<Other>,Asia,Western Asia,Iraq,(not set),9196717,YouTube Standards Zip Hoodie Black,YouTube,XS,Apparel,63.0,63.0,1,63.0,8965351293,(referral),10.0,Pay with credit card,NaN,referral,FedEx Ground,2
2,30013802.7072681119,1607454998290144,46.0,USD,desktop,Apple,Macintosh,None,Chrome,87.0,Asia,Western Asia,Iraq,(not set),9196909,Google Tudes Recycled Tee,Google,Single Option Only,New,24.0,24.0,1,24.0,3879740293,None,8.0,Pay with credit card,NaN,None,FedEx Ground,1
3,64559895.4607833037,1610481908254600,13.0,USD,desktop,Google,Web,None,Chrome,87.0,Americas,South America,Peru,(not set),9185002,Google Large Standard Journal Grey,Google,Single Option Only,Office,13.0,13.0,1,13.0,802523661,(referral),4.0,Pay with credit card,NaN,referral,FedEx Ground,1
4,53982377.9231749924,1605259651336262,48.0,USD,mobile,<Other>,Web,None,Chrome,87.0,Americas,South America,Peru,(not set),9195197,Google F/C Longsleeve Charcoal,Google,MD,Apparel,24.0,24.0,2,48.0,1464203493,None,NaN,Pay with credit card,NaN,None,None,13


In [7]:
data_purchase_uniquet_id_df.shape

(4786, 31)

In [72]:
from pandas_profiling import ProfileReport
profile = ProfileReport(data_purchase_uniquet_id_df, title="Pandas Profiling Report",minimal=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
feature_subset_query = """

SELECT user_pseudo_id,revenue,mobile_brand_name,operating_system,
browser,item_category,campaign,shipping_tier,
from ecomm_unnest_purchase.ecomm_raw_purchase_version03
WHERE transaction_id is not NULL
AND country = 'United States'
"""

query_job = client.query(feature_subset_query)
feature_subset_df = query_job.to_dataframe()
feature_subset_df.head(5)


,user_pseudo_id,revenue,mobile_brand_name,operating_system,browser,item_category,campaign,shipping_tier
0,22300841.9909876299,2.0,Apple,Web,Chrome,Office,<Other>,FedEx Ground
1,6266299.1262642458,6.0,Apple,Web,Chrome,Google,<Other>,FedEx Ground
2,85126752.5809296257,12.0,Apple,Macintosh,Chrome,Google,None,FedEx Ground
3,4650505.5425107709,6.0,Apple,Web,Safari,Google,None,None
4,16851116.6419517132,22.0,<Other>,Web,Chrome,Uncategorized Items,None,UPS Ground


In [13]:
feature_subset_df.shape

(2077, 8)

In [53]:
profile = ProfileReport(feature_subset_df, title="Pandas Profiling Report",minimal=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
feature_final_query = """
SELECT
    user_pseudo_id,
        case
        WHEN mobile_brand_name like 'Apple' then 'Apple'
        WHEN mobile_brand_name like 'Google' then 'Goole'
        else "Others"
        end as mobile_brand,
        case
        WHEN browser like 'Chrome' then 'Chrome'
        WHEN browser like 'Safari' then 'Safari'
        else "Others"
        end as browsers,
        case
        WHEN revenue < 50 then 'upto50'
        WHEN revenue >50 and revenue <100 then '50to100'
        WHEN revenue >100 and revenue <200 then '100to200'
        else '200UP'
        end as revenue_cat,
        operating_system as os,
        case
        WHEN item_category like 'Apparel' then 'Apparel'
        WHEN item_category like 'Shop by Brand' then 'Brand'
        WHEN item_category like 'New' then 'New'
        WHEN item_category like 'Drinkware' then 'Drinkware'
        WHEN item_category like 'Office' then 'Office'
        WHEN item_category like 'Campus Collection' then 'Campus'
        WHEN item_category like 'Uncategorized Items' then 'Uncat'
        else "Others"
        end as item_category,
        case
        WHEN campaign like '(referral)' then 'referral'
        WHEN campaign like '(organic)' then 'organic'
        WHEN campaign like '(direct)' then 'direct'
        else "Others"
        end as campaign,
        case
        WHEN shipping_tier like 'FedEx%' then 'fedx'
        WHEN shipping_tier like 'UPS%' then 'ups'
        else "Others"
        end as shipping,
FROM
    ecomm_unnest_purchase.ecomm_raw_purchase_version03
WHERE 
    transaction_id is not NULL
    AND country = 'United States'
"""

query_job = client.query(feature_final_query)
feature_final_df = query_job.to_dataframe()
feature_final_df.head(5)

,user_pseudo_id,mobile_brand,browsers,revenue_cat,os,item_category,campaign,shipping
0,22300841.9909876299,Apple,Chrome,upto50,Web,Office,Others,fedx
1,6266299.1262642458,Apple,Chrome,upto50,Web,Others,Others,fedx
2,85126752.5809296257,Apple,Chrome,upto50,Macintosh,Others,Others,fedx
3,4650505.5425107709,Apple,Safari,upto50,Web,Others,Others,Others
4,16851116.6419517132,Others,Chrome,upto50,Web,Uncat,Others,ups


In [56]:
profile = ProfileReport(feature_final_df, title="Pandas Profiling Report",minimal=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [57]:
# Creating a table with unnested data that can be used further 
query_create_newtable = """
CREATE table ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature
as
SELECT
    user_pseudo_id,
        case
        WHEN mobile_brand_name like 'Apple' then 'Apple'
        WHEN mobile_brand_name like 'Google' then 'Goole'
        else "Others"
        end as mobile_brand,
        case
        WHEN browser like 'Chrome' then 'Chrome'
        WHEN browser like 'Safari' then 'Safari'
        else "Others"
        end as browsers,
        case
        WHEN revenue < 50 then 'upto50'
        WHEN revenue >50 and revenue <100 then '50to100'
        WHEN revenue >100 and revenue <200 then '100to200'
        else '200UP'
        end as revenue_cat,
        operating_system as os,
        case
        WHEN item_category like 'Apparel' then 'Apparel'
        WHEN item_category like 'Shop by Brand' then 'Brand'
        WHEN item_category like 'New' then 'New'
        WHEN item_category like 'Drinkware' then 'Drinkware'
        WHEN item_category like 'Office' then 'Office'
        WHEN item_category like 'Campus Collection' then 'Campus'
        WHEN item_category like 'Uncategorized Items' then 'Uncat'
        else "Others"
        end as item_category,
        case
        WHEN campaign like '(referral)' then 'referral'
        WHEN campaign like '(organic)' then 'organic'
        WHEN campaign like '(direct)' then 'direct'
        else "Others"
        end as campaign,
        case
        WHEN shipping_tier like 'FedEx%' then 'fedx'
        WHEN shipping_tier like 'UPS%' then 'ups'
        else "Others"
        end as shipping,
FROM
    ecomm_unnest_purchase.ecomm_raw_purchase_version03
WHERE 
    transaction_id is not NULL
    AND country = 'United States'
    """

client = bigquery.Client()
query_job = client.query(query_create_newtable)

In [61]:
%%bigquery
select * 
from ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature 

Downloading: 100%|██████████| 2077/2077 [00:01<00:00, 1818.06rows/s]


,user_pseudo_id,mobile_brand,browsers,revenue_cat,os,item_category,campaign,shipping
0,69784832.7107601315,Goole,Chrome,upto50,Web,New,Others,ups
1,32231127.3526233643,Apple,Safari,upto50,Web,New,Others,Others
2,1405049.6223660087,Others,Others,50to100,Web,New,Others,fedx
3,69784832.7107601315,Goole,Chrome,50to100,Web,New,Others,ups
4,74016002.6019650209,Goole,Chrome,200UP,Web,New,Others,Others
...,...,...,...,...,...,...,...,...
2072,5556391.0179080945,Apple,Chrome,upto50,Macintosh,Drinkware,Others,fedx
2073,70092642.2710325772,Apple,Chrome,upto50,Macintosh,Drinkware,Others,fedx
2074,1602466.3255340626,Apple,Safari,upto50,Macintosh,Drinkware,Others,Others
2075,65224534.5204315996,Apple,Chrome,50to100,Macintosh,Drinkware,direct,fedx


In [74]:
%%bigquery
select * 
from bq-test-01-338016.ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature 

Downloading: 100%|██████████| 2077/2077 [00:01<00:00, 1977.29rows/s]


,user_pseudo_id,mobile_brand,browsers,revenue_cat,os,item_category,campaign,shipping
0,69784832.7107601315,Goole,Chrome,upto50,Web,New,Others,ups
1,32231127.3526233643,Apple,Safari,upto50,Web,New,Others,Others
2,1405049.6223660087,Others,Others,50to100,Web,New,Others,fedx
3,69784832.7107601315,Goole,Chrome,50to100,Web,New,Others,ups
4,74016002.6019650209,Goole,Chrome,200UP,Web,New,Others,Others
...,...,...,...,...,...,...,...,...
2072,5556391.0179080945,Apple,Chrome,upto50,Macintosh,Drinkware,Others,fedx
2073,70092642.2710325772,Apple,Chrome,upto50,Macintosh,Drinkware,Others,fedx
2074,1602466.3255340626,Apple,Safari,upto50,Macintosh,Drinkware,Others,Others
2075,65224534.5204315996,Apple,Chrome,50to100,Macintosh,Drinkware,direct,fedx


In [80]:
%%bigquery
CREATE MODEL
  `ecomm_unnest_purchase.mymodel`
OPTIONS
  ( MODEL_TYPE='KMEANS',
    NUM_CLUSTERS=4 ) AS
SELECT
  *
FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1897.30query/s]                        


""


In [ ]:
CREATE OR REPLACE MODEL
  `ecomm_unnest_purchase.mymodel`
OPTIONS
  ( MODEL_TYPE='KMEANS',
    KMEANS_INIT_METHOD = 'KMEANS++',
    STANDARDIZE_FEATURES = TRUE) AS
SELECT
  * EXCEPT (user_pseudo_id)
FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`

In [98]:

def makeModel (num_clusters, model_name, auto_k=False):
    dataset_id = "ecomm_unnest_purchase"
    feature_table="ecomm_raw_purchase_finalfeature"
    if not auto_k:
        query = """
        CREATE OR REPLACE MODEL
        `{dataset_id}.{model_name}`
        OPTIONS
         (  NUM_CLUSTERS = {num_clusters},
            MODEL_TYPE='KMEANS',
            KMEANS_INIT_METHOD ='RANDOM',
            STANDARDIZE_FEATURES = TRUE
         ) AS
        SELECT
              * EXCEPT (user_pseudo_id)
        FROM `{dataset_id}.{feature_table}`
    """.format(dataset_id=dataset_id,model_name=model_name,
               num_clusters=num_clusters,feature_table=feature_table)
    else:
        query = """
        CREATE OR REPLACE MODEL
        `{dataset_id}.{model_name}`
        OPTIONS
         (  MODEL_TYPE='KMEANS++',
            KMEANS_INIT_METHOD ='RANDOM',
            STANDARDIZE_FEATURES = TRUE
         ) AS
        SELECT
              * EXCEPT (user_pseudo_id)
        FROM `{dataset_id}.{feature_table}`
        """.format(dataset_id=dataset_id,model_name=model_name,
               feature_table=feature_table)
    print(query)
    job_config = bigquery.QueryJobConfig()
    client.query(query, job_config=job_config)  # Make an API request.

In [99]:
makeModel(3, "mymodel2",auto_k=True)


        CREATE OR REPLACE MODEL
        `ecomm_unnest_purchase.mymodel2`
        OPTIONS
         (  MODEL_TYPE='KMEANS++',
            KMEANS_INIT_METHOD ='RANDOM',
            STANDARDIZE_FEATURES = TRUE
         ) AS
        SELECT
              * EXCEPT (user_pseudo_id)
        FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`
        


In [101]:
client.list_models(dataset_id)

In [100]:
for model in client.list_models(dataset_id):
    print(model)

Model(reference=ModelReference(project_id='bq-test-01-338016', dataset_id='ecomm_unnest_purchase', model_id='mymodel'))
Model(reference=ModelReference(project_id='bq-test-01-338016', dataset_id='ecomm_unnest_purchase', model_id='test'))


In [103]:
%%bigquery
SELECT 
*
FROM ML.EVALUATE(MODEL `ecomm_unnest_purchase.mymodel`)

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.10s/rows]


,davies_bouldin_index,mean_squared_distance
0,3.090183,3.446731


In [105]:
%%bigquery
DECLARE x INT64 DEFAULT 0;
WHILE x < 10 
DO
  SET x = x + 1;
END WHILE;
SELECT x;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.08s/rows]


,x
0,10


In [115]:
%%bigquery
DECLARE num_cluster INT64 DEFAULT 1;
WHILE num_cluster < 3

DO
    CREATE OR REPLACE MODEL
    `ecomm_unnest_purchase.name`
    OPTIONS
    ( MODEL_TYPE='KMEANS',
    NUM_CLUSTERS=num_cluster ) AS
    SELECT
    *
    FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`;
    SET num_cluster = num_cluster + 1;
END WHILE;

Executing query with job ID: 07664f28-d9cc-419f-b1ed-6329c99f4e6f
Query executing: 0.33s


ERROR:
 400 Query error: Could not cast literal "mymodel_" to type DATE at [3:12]

(job ID: 07664f28-d9cc-419f-b1ed-6329c99f4e6f)

                   -----Query Job SQL Follows-----                    

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:DECLARE num_cluster INT64 DEFAULT 1;
   2:DECLARE name STRING;
   3:SET name = 'mymodel_'+num_cluster;
   4:WHILE num_cluster < 3
   5:
   6:DO
   7:    CREATE OR REPLACE MODEL
   8:    `ecomm_unnest_purchase.name`
   9:    OPTIONS
  10:    ( MODEL_TYPE='KMEANS',
  11:    NUM_CLUSTERS=num_cluster ) AS
  12:    SELECT
  13:    *
  14:    FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`;
  15:    SET num_cluster = num_cluster + 1;
  16:END WHILE;
    |    .    |    .    |    .    |    .    |    .    |    .    |


In [121]:
%%bigquery
DECLARE num_cluster INT64 DEFAULT 1;
DECLARE model_name STRING DEFAULT 'model_';
WHILE num_cluster < 3 DO
    CREATE or REPLACE MODEL
    `ecomm_unnest_purchase.model_name`
    OPTIONS
    ( MODEL_TYPE='KMEANS',
    NUM_CLUSTERS=num_cluster ) AS
    SELECT
    *
    FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`;
    SET num_cluster = num_cluster + 1;
    SET model_name = CONCAT(model_name , CAST(num_cluster AS STRING));
END WHILE;

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 966.21query/s] 


""


In [131]:
#Listing the Model
client = bigquery.Client()
for model in client.list_models(dataset_id):
    print(model.model_id)

model_
model_4
model_name
mymodel
test


In [134]:
#Deleting the model 
client = bigquery.Client()
for model in client.list_models(dataset_id):
    client.delete_model(dataset_id+'.'+model.model_id)

In [135]:
#Listing the Model
client = bigquery.Client()
for model in client.list_models(dataset_id):
    print(model.model_id)

In [106]:
%%bigquery
DECLARE name STRING;
SET name = '_5_18_20';
EXECUTE IMMEDIATE format("""
DECLARE x INT64 DEFAULT 0;
WHILE x < 2 
DO
    CREATE MODEL
    `ecomm_unnest_purchase.mymodel_%s`
    OPTIONS
    ( MODEL_TYPE='KMEANS',
    NUM_CLUSTERS=x ) AS
    SELECT
    *
    FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`
    SET x = x + 1;
END WHILE;
SELECT x;
""",x)

Executing query with job ID: e8738ec6-8819-45b9-ae6e-7d0ea0765722
Query executing: 0.25s


ERROR:
 400 Query error: Unrecognized name: x at [16:5]

(job ID: e8738ec6-8819-45b9-ae6e-7d0ea0765722)

                   -----Query Job SQL Follows-----                   

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:EXECUTE IMMEDIATE format("""
   2:DECLARE x INT64 DEFAULT 0;
   3:WHILE x < 2 
   4:DO
   5:    CREATE MODEL
   6:    `ecomm_unnest_purchase.mymodel_%s`
   7:    OPTIONS
   8:    ( MODEL_TYPE='KMEANS',
   9:    NUM_CLUSTERS=x ) AS
  10:    SELECT
  11:    *
  12:    FROM `ecomm_unnest_purchase.ecomm_raw_purchase_finalfeature`
  13:    SET x = x + 1;
  14:END WHILE;
  15:SELECT x;
  16:""",x)
    |    .    |    .    |    .    |    .    |    .    |    .    |
